# Assignment 4: Candidate Moves

- Łukasz Andryszewski 151930
- Filip Firkowski 151946

Link to the repository is: https://github.com/lucapl/Evolutionary-Computations.

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd

from utils import *
from plotting import *
pd.set_option('display.max_colwidth', None)

## Description of a problem:

The problem is about selecting exactly 50% of the nodes to form a Hamiltonian cycle that minimizes the total distance of the path and the total cost of the selected nodes.

In this report the candidate edges mechanism is evaluated. The goal being to speed up the steepest local search algorithm.

## Pseudocode of all implemented algorithms

<style>
  .no-page-break {
    page-break-inside: avoid;
  }
</style>

<div class="no-page-break">
  <h3>Steepest Local Search with Move Evaluation Deltas:</h3>
  <pre>
generate_random_solution()

initialize improving_moves as empty

while solution_is_improving:
  for move in improving_moves:
    if removed_edges_no_longer_exist_in_solution(move):
      remove_move_from_LM(move)
    elif edges_exist_with_different_relative_direction(move):
      keep_move_in_LM()
    elif edges_exist_with_same_relative_direction(move):
      apply_move_to_solution(move)
      remove_move_from_LM(move)

  candidate_moves = calculate_candidate_moves()
  candidate_moves = add_inverted_edge_moves(candidate_moves)

  best_move = find_best_move(candidate_moves)

  if best_move:
    apply_move_to_solution(best_move)
    improving_moves.add(best_move)

return solution
  </pre>
</div>

<div class="no-page-break">
  <h3>Calculating Candidate Moves with Inverted Edges:</h3>
  <pre>
for city_i in cities:
  for city_j in cities:
    potential_moves.add(move(city_i, city_j))
    potential_moves.add(inverted_move(city_j, city_i))

  sort potential_moves
  candidate_moves.add(potential_moves.first_n(10))
  </pre>
</div>


The complexity of calculating the candidate moves is $O(n^2)$, where n is the number of cities. However these are calculated only once.

Here the neighbourhood of each city is calculated not only by the distance to each neighbour but also the neighbours cost.

<style>
  table {
    width: 100%;
    table-layout: fixed;
    word-wrap: break-word;
  }
</style>

## Results of a computational experiments

In [ ]:
solver_types = ["lmSearch-Random"]
instances = ['A', 'B']

folder_path = '../out5'
all_json_data = load_json_files_from_folder(folder_path)

table, best_solutions = get_best_solutions_and_vertical_table(solver_types,instances,all_json_data)

In [ ]:
localSearch = read_html('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Method</th>\n      <th>Instance A</th>\n      <th>Instance B</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>localSearchSteepestNodesRandom</td>\n      <td>88106.7 (79865.0-98435.0)</td>\n      <td>63238.6 (56390.0-72239.0)</td>\n    </tr>\n    <tr>\n      <td>localSearchSteepestEdgesRandom</td>\n      <td>74001.6 (71311.0-77763.0)</td>\n      <td>48249.9 (45981.0-51599.0)</td>\n    </tr>\n  </tbody>\n</table>')

display_html(pd.concat([localSearch,table]),False)

<p style="page-break-after:always;"></p>

Table containing elapsed time in ms.

In [ ]:
timeTable, _ = get_best_solutions_and_vertical_table(solver_types,instances,all_json_data,"elapsed time")
localSearchTime = read_html('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Method</th>\n      <th>Instance A</th>\n      <th>Instance B</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>localSearchSteepestNodesRandom</td>\n      <td>172.0 (136.8-430.3)</td>\n      <td>169.2 (129.4-273.2)</td>\n    </tr>\n    <tr>\n      <td>localSearchSteepestEdgesRandom</td>\n      <td>118.9 (104.8-145.0)</td>\n      <td>139.6 (105.2-496.6)</td>\n    </tr>\n  </tbody>\n</table>')

display_html(pd.concat([localSearchTime,timeTable]),False)

## Best solutions:

The following solutions were checked with the solution checker.

In [ ]:
print_solutions(solver_types,instances,best_solutions)

## 2D visualizations:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

coordinates = {
    'A': load_coordinates_from_csv('../src/main/resources/instances/TSPA.csv'),
    'B': load_coordinates_from_csv('../src/main/resources/instances/TSPB.csv')
}

plot_all(solver_types, instances, coordinates, best_solutions)

<p style="page-break-after:always;"></p>

# Conclusions:

Overall, implementing move evaluation deltas in the steepest local search significantly improves the algorithm's efficiency. By reusing previously evaluated moves, the algorithm avoids redundant calculations, speeding up the search process.

However, the impact on solution quality is minimal compared to the standard steepest local search. There is a trade-off between algorithm efficiency and the quality of the solutions produced.

The performance is largely dependent on the number of improving moves in the list and how frequently new candidate moves are evaluated. If the list of improving moves is large or if moves are evaluated at each step, the algorithm performs similarly to the baseline steepest local search with random starting solutions, which does not utilize these mechanisms.

In summary, while the move evaluation delta mechanism increases efficiency, the solution quality remains close to the baseline approach. The choice between these methods depends on the specific needs for speed versus solution quality.